# एक पशु विशेषज्ञ प्रणाली को लागू करना

[AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) से एक उदाहरण।

इस नमूने में, हम कुछ शारीरिक लक्षणों के आधार पर एक पशु का निर्धारण करने के लिए एक सरल ज्ञान-आधारित प्रणाली लागू करेंगे। प्रणाली को निम्नलिखित AND-OR पेड़ द्वारा दर्शाया जा सकता है (यह पूरे पेड़ का एक हिस्सा है, हम आसानी से कुछ और नियम जोड़ सकते हैं):

![](../../../../../../translated_images/hi/AND-OR-Tree.5592d2c70187f283.webp)


## हमारे अपने विशेषज्ञ प्रणालियों का शेल बैकवर्ड इनफेरेंस के साथ

आइए उत्पादन नियमों पर आधारित ज्ञान प्रतिनिधित्व के लिए एक सरल भाषा परिभाषित करने का प्रयास करें। हम नियमों को परिभाषित करने के लिए कीवर्ड के रूप में Python कक्षाओं का उपयोग करेंगे। मूल रूप से तीन प्रकार की कक्षाएं होंगी:
* `Ask` एक प्रश्न को दर्शाती है जिसे उपयोगकर्ता से पूछा जाना आवश्यक है। इसमें संभावित उत्तरों का सेट होता है।
* `If` एक नियम को दर्शाती है, और यह केवल नियम की सामग्री संग्रहीत करने के लिए एक व्याकरणिक शर्करा है।
* `AND`/`OR` कक्षाएं पेड़ की AND/OR शाखाओं को दर्शाने के लिए हैं। वे केवल अंदर के तर्कों की सूची संग्रहीत करती हैं। कोड को सरल बनाने के लिए, सारी कार्यक्षमता पैरेंट क्लास `Content` में परिभाषित की गई है।


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

हमारे सिस्टम में, वर्किंग मेमोरी में **तथ्यों** की सूची होगी जो **विशेषता-मूल्य जोड़े** के रूप में होगी। ज्ञान आधार को एक बड़े शब्दकोश के रूप में परिभाषित किया जा सकता है जो क्रियाओं (नए तथ्य जिन्हें वर्किंग मेमोरी में सम्मिलित किया जाना चाहिए) को शर्तों से मैप करता है, जिन्हें AND-OR अभिव्यक्तियों के रूप में व्यक्त किया गया है। साथ ही, कुछ तथ्यों से `पूछा` भी जा सकता है।


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

बैकवर्ड इनफेरेंस करने के लिए, हम `Knowledgebase` क्लास परिभाषित करेंगे। इसमें शामिल होंगे:
* वर्किंग `memory` - एक डिक्शनरी जो एट्रीब्यूट्स को मानों से मैप करती है
* नॉलेजबेस `rules` उपर बताए गए फॉर्मेट में

दो मुख्य मेथड हैं:
* `get` किसी एट्रीब्यूट का मान प्राप्त करने के लिए, आवश्यक होने पर इनफ़ेरेंस करता है। उदाहरण के लिए, `get('color')` किसी कलर स्लॉट का मान प्राप्त करेगा (यदि ज़रूरी हो तो पूछेगा, और बाद में उपयोग के लिए वर्किंग मेमोरी में मान संग्रहीत करेगा)। यदि हम `get('color:blue')` पूछें, तो यह कलर के लिए पूछेगा, और फिर कलर के आधार पर `y`/`n` मान लौटाएगा।
* `eval` वास्तविक इनफ़ेरेंस करता है, यानि AND/OR ट्री को ट्रैवर्स करता है, सब-गोल्स का मूल्यांकन करता है, आदि।


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

अब आइए अपने पशु ज्ञानकोष को परिभाषित करें और परामर्श करें। ध्यान दें कि यह कॉल आपसे प्रश्न पूछेगा। आप हाँ-नहीं प्रश्नों के लिए `y`/`n` टाइप करके उत्तर दे सकते हैं, या लंबे बहुविकल्पीय उत्तरों वाले प्रश्नों के लिए संख्या (0..N) निर्दिष्ट कर सकते हैं।


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## फॉरवर्ड इनफेरेंस के लिए Experta का उपयोग करना

अगले उदाहरण में, हम ज्ञान प्रतिनिधित्व के लिए लाइब्रेरियों में से एक, [Experta](https://github.com/nilp0inter/experta) का उपयोग करके फॉरवर्ड इनफेरेंस लागू करने का प्रयास करेंगे। **Experta** एक लाइब्रेरी है जो Python में फॉरवर्ड इनफेरेंस सिस्टम बनाने के लिए बनाई गई है, जिसे पारंपरिक पुराने सिस्टम [CLIPS](http://www.clipsrules.net/index.html) के समान डिज़ाइन किया गया है।

हम बिना किसी बड़ी समस्या के फॉरवर्ड चेनिंग स्वयं भी लागू कर सकते थे, लेकिन सामान्य कार्यान्वयन आमतौर पर बहुत कुशल नहीं होते। अधिक प्रभावी नियम मिलान के लिए एक विशेष एल्गोरिदम [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) का उपयोग किया जाता है।


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

हम अपनी प्रणाली को `KnowledgeEngine` की एक उपवर्ग के रूप में परिभाषित करेंगे। प्रत्येक नियम को एक अलग फ़ंक्शन के रूप में परिभाषित किया जाता है जिसमें `@Rule` एनोटेशन होता है, जो निर्दिष्ट करता है कि नियम कब सक्रिय होना चाहिए। नियम के अंदर, हम `declare` फ़ंक्शन का उपयोग करके नए तथ्य जोड़ सकते हैं, और उन तथ्यों को जोड़ने से आगे की अनुमान इंजन द्वारा कुछ और नियमों को सक्रिय किया जाएगा।


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

एक बार जब हमने एक नॉलेजबेस परिभाषित कर दिया, तो हम अपनी कार्यशील मेमोरी को कुछ प्रारंभिक तथ्यों से भरते हैं, और फिर इनफेरेंस करने के लिए `run()` मेथड को कॉल करते हैं। आप परिणामस्वरूप देख सकते हैं कि नए निष्कर्षित तथ्य कार्यशील मेमोरी में जोड़े जाते हैं, जिसमें जानवर के बारे में अंतिम तथ्य भी शामिल है (यदि हमने सभी प्रारंभिक तथ्य सही ढंग से सेट किए हैं)।


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:  
इस दस्तावेज़ का अनुवाद AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके किया गया है। यद्यपि हम सटीकता के लिए प्रयासरत हैं, कृपया ध्यान रखें कि स्वचालित अनुवादों में त्रुटियां या अशुद्धियां हो सकती हैं। मूल दस्तावेज़ अपनी मूल भाषा में आधिकारिक स्रोत माना जाए। महत्वपूर्ण जानकारी के लिए पेशेवर मानव अनुवाद की सिफारिश की जाती है। हम इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए जिम्मेदार नहीं हैं।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
